# ES → BBF Salesforce Service Migration

This notebook migrates Order records from ES Salesforce to Service__c in BBF Salesforce.

## Prerequisites
- **Account migration must be completed first** (Accounts have BBF_New_Id__c populated)
- **BAN__c migration must be completed first** (Billing_Invoice__c has BBF_New_Id__c populated)
- **Location__c migration must be completed first** (Address__c has BBF_New_Id__c populated)
- ES Order records must have BBF_New_Id__c field created

## Object Mapping
- **ES Source:** Order (Salesforce Standard Object with custom fields)
- **BBF Target:** Service__c (Custom Object)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query Order from ES where:
   - Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
   - Project_Group__c NOT LIKE '%PA MARKET DECOM%'
   - Service_Order_Record_Type__c = 'Service Order Agreement' 
   - Parent BAN (Billing_Invoice__c) has BBF_New_Id__c populated (BAN already migrated)
   - Parent Account has BBF_New_Id__c populated (Account already migrated)
   - Address_A__c has BBF_New_Id__c populated (Location already migrated)
   - BBF_New_Id__c is empty (not yet migrated)
3. Transform ES Order for BBF Service__c schema:
   - Map Billing_Invoice__c → BBF BAN ID via ES Billing_Invoice__r.BBF_New_Id__c (MASTER-DETAIL REQUIRED)
   - Map AccountId → BBF Account ID via ES Account.BBF_New_Id__c
   - Map Address_A__c → BBF Location ID via ES Address__r.BBF_New_Id__c 
   - Map Address_Z__c → BBF Location ID via ES Address__r.BBF_New_Id__c
   - Set required fields: TSP__c = False, busUnit__c = 'EVS'
   - Add ES_Legacy_ID__c = ES Order.Id (for tracking)
4. Insert Service__c to BBF Salesforce
5. Update ES Order with BBF_New_Id__c = BBF Service.Id
6. Create ID mapping: ES Order ID → BBF Service ID
7. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF Service__c:** `ES_Legacy_ID__c` stores original ES Order ID
- Text(18), External ID, Unique

**In ES Order:**
- `BBF_New_Id__c` stores new BBF Service ID after migration (Text 18)

## Safety
- `TEST_MODE = True` by default (limits to 10 Services)
- Only migrates Orders with Status = Activated/Suspended/Disconnect in Progress
- Skips Orders where parent BAN not yet migrated
- Skips Orders where parent Account not yet migrated
- Skips Orders where Location not yet migrated
- Skips Orders already migrated (BBF_New_Id__c populated)
- Skips PA MARKET DECOM orders

In [ ]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL Services
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Service Owner - Set all migrated Services to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as Account/Contact/BAN/Location migration

# 🏢 Default Business Unit - Required picklist field in BBF
# ES records being migrated should be assigned to EVS business unit
DEFAULT_BUS_UNIT = "EVS"  # EverStream business unit in BBF

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_service_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Default Business Unit: {DEFAULT_BUS_UNIT}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

In [ ]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

In [ ]:
# === QUERY ES ORDER (Services) ===
# Query fields needed for Day 1 migration including Location lookups
# BBF Service__c: Name is AUTONUMBER (don't set), no OwnerId field

print("\n" + "=" * 80)
print("QUERYING ES ORDER (Services)")
print("=" * 80)

# Query includes:
# - Id: for ES_Legacy_ID__c tracking
# - Billing_Invoice__r.BBF_New_Id__c: for master-detail to BAN__c
# - Address_A__r.BBF_New_Id__c: for A_Location__c lookup
# - Address_Z__r.BBF_New_Id__c: for Z_Location__c lookup
query = """SELECT Id, 
    Billing_Invoice__r.BBF_New_Id__c,
    Address_A__c, Address_A__r.BBF_New_Id__c,
    Address_Z__c, Address_Z__r.BBF_New_Id__c
FROM Order
WHERE Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
AND (Project_Group__c = null OR (NOT Project_Group__c LIKE '%PA MARKET DECOM%'))
AND Service_Order_Record_Type__c = 'Service Order Agreement'
AND Billing_Invoice__r.BBF_New_Id__c != null
AND Billing_Invoice__r.BBF_New_Id__c != ''
AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')"""

# Add limit for test mode
if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query:\n{query}")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_orders_raw = result["records"]

print(f"✅ Found {len(es_orders_raw)} Order records from ES query")

# =============================================================================
# DUPLICATE PREVENTION: Check BBF for already-migrated records
# If record exists in BBF, get its Id to update ES BBF_New_Id__c
# =============================================================================
print("\n" + "-" * 80)
print("DUPLICATE PREVENTION: Checking BBF for existing ES_Legacy_ID__c...")
print("-" * 80)

bbf_existing_query = """
SELECT Id, ES_Legacy_ID__c 
FROM Service__c 
WHERE ES_Legacy_ID__c != null
"""
bbf_existing_result = bbf_sf.query_all(bbf_existing_query)

# Build lookup: ES_Legacy_ID__c -> BBF Id
existing_bbf_lookup = {r['ES_Legacy_ID__c']: r['Id'] for r in bbf_existing_result['records']}

print(f"   Found {len(existing_bbf_lookup)} Service__c records already in BBF")

# Separate: records to migrate vs records that need ES BBF_New_Id__c sync
es_orders = []
es_needs_sync = []  # Records that exist in BBF but ES.BBF_New_Id__c is null

for order in es_orders_raw:
    if order['Id'] in existing_bbf_lookup:
        # Already in BBF - need to sync ES.BBF_New_Id__c
        es_needs_sync.append({
            'es_id': order['Id'],
            'bbf_id': existing_bbf_lookup[order['Id']]
        })
    else:
        # Not in BBF - need to migrate
        es_orders.append(order)

print(f"   Records to migrate (not in BBF): {len(es_orders)}")
print(f"   Records to sync (in BBF, ES.BBF_New_Id__c missing): {len(es_needs_sync)}")

# Sync ES.BBF_New_Id__c for records that already exist in BBF
if len(es_needs_sync) > 0:
    print(f"\n📌 Syncing {len(es_needs_sync)} ES Order BBF_New_Id__c values...")
    
    sync_success = 0
    sync_failed = 0
    
    # Use individual updates to avoid @future call limits
    for item in es_needs_sync:
        try:
            es_sf.Order.update(item['es_id'], {'BBF_New_Id__c': item['bbf_id']})
            sync_success += 1
        except Exception as e:
            sync_failed += 1
            if sync_failed <= 5:
                print(f"     ❌ {item['es_id']}: {e}")
    
    print(f"   ✅ Synced: {sync_success}")
    print(f"   ❌ Failed to sync: {sync_failed}")

print(f"\n✅ {len(es_orders)} Order records to migrate (after duplicate check)")

if len(es_orders) > 0:
    sample = es_orders[0]
    addr_a = sample.get("Address_A__r") or {}
    addr_z = sample.get("Address_Z__r") or {}
    print(f"\nSample Order:")
    print(f"  ID:              {sample['Id']}")
    print(
        f"  BBF BAN ID:      {sample.get('Billing_Invoice__r', {}).get('BBF_New_Id__c', 'N/A')}"
    )
    print(f"  BBF A_Location:  {addr_a.get('BBF_New_Id__c', 'N/A')}")
    print(f"  BBF Z_Location:  {addr_z.get('BBF_New_Id__c', 'N/A')}")
else:
    print("\n✅ All Order records have been migrated (or no eligible records found)!")

In [ ]:
# === TRANSFORM FOR BBF SERVICE__c ===
# Day 1 Migration: Required fields + Location lookups
# - Name: AUTONUMBER (don't set)
# - OwnerId: NOT ON SERVICE__c
# - Billing_Account_Number__c: Master-Detail (REQUIRED)
# - A_Location__c, Z_Location__c: Optional lookups

print("\n" + "=" * 80)
print("TRANSFORMING ORDER FOR BBF SERVICE__c")
print("=" * 80)

bbf_services = []
skipped_no_bbf_ban = []
location_stats = {"a_location": 0, "z_location": 0}

for es_order in es_orders:
    # Get the BBF BAN ID from the parent Billing_Invoice__c (MASTER-DETAIL REQUIRED)
    bbf_ban_id = None
    if es_order.get("Billing_Invoice__r") and es_order["Billing_Invoice__r"].get(
        "BBF_New_Id__c"
    ):
        bbf_ban_id = es_order["Billing_Invoice__r"]["BBF_New_Id__c"]

    # Safety check: Skip if no BBF BAN ID (MASTER-DETAIL is BLOCKING)
    if not bbf_ban_id:
        skipped_no_bbf_ban.append(
            {
                "es_id": es_order["Id"],
                "es_ban_id": es_order.get("Billing_Invoice__c"),
                "reason": "No BBF BAN ID found - Master-Detail is required",
            }
        )
        continue

    # Get Location IDs if available
    bbf_a_location = None
    bbf_z_location = None

    addr_a = es_order.get("Address_A__r") or {}
    if addr_a.get("BBF_New_Id__c"):
        bbf_a_location = addr_a["BBF_New_Id__c"]
        location_stats["a_location"] += 1

    addr_z = es_order.get("Address_Z__r") or {}
    if addr_z.get("BBF_New_Id__c"):
        bbf_z_location = addr_z["BBF_New_Id__c"]
        location_stats["z_location"] += 1

    # =========================================================================
    # BBF Service__c - Fields to set
    # Name = Autonumber (don't set)
    # No OwnerId field on Service__c
    # =========================================================================
    bbf_service = {
        # 🔴 REQUIRED: Master-Detail to BAN__c
        "Billing_Account_Number__c": bbf_ban_id,
        # 🔗 Tracking field
        "ES_Legacy_ID__c": es_order["Id"],
    }

    # Optional: Location lookups (if migrated)
    if bbf_a_location:
        bbf_service["A_Location__c"] = bbf_a_location
    if bbf_z_location:
        bbf_service["Z_Location__c"] = bbf_z_location

    bbf_services.append(bbf_service)

print(f"✅ Transformed {len(bbf_services)} Services")
print(f"\n   FIELDS SET:")
print(f"   - Billing_Account_Number__c (Master-Detail to BAN__c)")
print(f"   - ES_Legacy_ID__c (tracking)")
print(f"   - A_Location__c: {location_stats['a_location']} Services")
print(f"   - Z_Location__c: {location_stats['z_location']} Services")
print(f"\n   Note: Name is Autonumber, OwnerId not on Service__c")
print(f"   Note: Account__c will be populated post-insert from BAN relationship")

if len(skipped_no_bbf_ban) > 0:
    print(f"\n⚠️  Skipped {len(skipped_no_bbf_ban)} Services (no BBF BAN ID - BLOCKING)")
    for skip in skipped_no_bbf_ban[:5]:
        print(f"   - ES Order: {skip['es_id']}")

In [ ]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING SERVICES TO BBF")
print("=" * 80)

if len(bbf_services) == 0:
    print("⚠️  No Services to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_services)} Services using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Service__c.insert(bbf_services)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_services[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "bbf_ban_id": bbf_services[i]["Billing_Account_Number__c"],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_services[i]["ES_Legacy_ID__c"],
                        "errors": result["errors"],
                        "bbf_ban_id": bbf_services[i]["Billing_Account_Number__c"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} Services")
        print(f"❌ Failed to insert: {len(failed_inserts)} Services")

        if len(failed_inserts) > 0:
            print(f"\nFailed Services (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - ES ID: {item['es_id']}")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES Order ID:     {sample['es_id']}")
            print(f"  BBF Service ID:  {sample['bbf_id']}")
            print(f"  BBF BAN ID:      {sample['bbf_ban_id']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []

In [ ]:
# === UPDATE ES WITH BBF IDS (BATCHED) ===
# ES Order triggers have @future methods - limit 50 per transaction
# Batch updates in small chunks to avoid governor limits

import time

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS (BATCHED)")
print("=" * 80)

# Batch size for ES updates - small to avoid future call limits
ES_UPDATE_BATCH_SIZE = 200
ES_UPDATE_DELAY_SECONDS = 2  # Delay between batches

if len(successful_inserts) == 0:
    print("⚠️  No Services to update in ES")
    es_update_results = []
    es_update_success = 0
    es_update_failed = 0
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    total_updates = len(es_updates)
    num_batches = (total_updates + ES_UPDATE_BATCH_SIZE - 1) // ES_UPDATE_BATCH_SIZE

    print(f"Updating {total_updates} Order records in ES...")
    print(f"   - Batch size: {ES_UPDATE_BATCH_SIZE} records")
    print(f"   - Total batches: {num_batches}")
    print(f"   - Delay between batches: {ES_UPDATE_DELAY_SECONDS}s")
    print(f"   - Setting BBF_New_Id__c = BBF Service ID")
    print()

    es_update_results = []
    es_update_success = 0
    es_update_failed = 0

    for batch_num in range(num_batches):
        start_idx = batch_num * ES_UPDATE_BATCH_SIZE
        end_idx = min(start_idx + ES_UPDATE_BATCH_SIZE, total_updates)
        batch = es_updates[start_idx:end_idx]

        print(
            f"   Batch {batch_num + 1}/{num_batches}: records {start_idx + 1}-{end_idx}...",
            end=" ",
        )

        try:
            # Use regular update for small batches (not bulk)
            batch_results = []
            for update in batch:
                try:
                    es_sf.Order.update(
                        update["Id"], {"BBF_New_Id__c": update["BBF_New_Id__c"]}
                    )
                    batch_results.append({"success": True, "id": update["Id"]})
                    es_update_success += 1
                except Exception as e:
                    batch_results.append(
                        {"success": False, "id": update["Id"], "errors": str(e)}
                    )
                    es_update_failed += 1

            es_update_results.extend(batch_results)
            batch_success = sum(1 for r in batch_results if r["success"])
            print(f"✅ {batch_success}/{len(batch)} succeeded")

        except Exception as e:
            print(f"❌ Batch error: {e}")
            for update in batch:
                es_update_results.append(
                    {"success": False, "id": update["Id"], "errors": str(e)}
                )
                es_update_failed += 1

        # Delay between batches (except last one)
        if batch_num < num_batches - 1:
            time.sleep(ES_UPDATE_DELAY_SECONDS)

    print()
    print(f"✅ Successfully updated: {es_update_success} Order records in ES")
    print(f"❌ Failed to update: {es_update_failed} Order records in ES")

    if es_update_failed > 0:
        print(
            "\n⚠️  Some ES updates failed. You can run the 'True-Up' cell below to retry."
        )
        print("   Failed updates (first 10):")
        fail_count = 0
        for r in es_update_results:
            if not r["success"] and fail_count < 10:
                print(f"     - {r['id']}: {r.get('errors', 'Unknown error')}")
                fail_count += 1

In [12]:
# === TRUE-UP: SYNC ES BBF_New_Id__c FROM BBF ES_Legacy_ID__c ===
# This cell can be run independently to fill in any missing BBF_New_Id__c values
# It queries BBF Services with ES_Legacy_ID__c and updates the matching ES Orders
# Useful if the ES write-back failed or was skipped

import time

print("\n" + "=" * 80)
print("TRUE-UP: SYNC ES BBF_New_Id__c FROM BBF ES_Legacy_ID__c")
print("=" * 80)

# Configuration
TRUEUP_BATCH_SIZE = 200
TRUEUP_DELAY_SECONDS = 2

# Step 1: Query BBF Services that have ES_Legacy_ID__c
print("\n📌 Step 1: Query BBF Services with ES_Legacy_ID__c...")
bbf_query = """SELECT Id, ES_Legacy_ID__c 
FROM Service__c 
WHERE ES_Legacy_ID__c != null AND ES_Legacy_ID__c != ''"""

bbf_result = bbf_sf.query_all(bbf_query)
bbf_services = bbf_result["records"]
print(f"   Found {len(bbf_services)} BBF Services with ES_Legacy_ID__c")

if len(bbf_services) == 0:
    print("\n⚠️  No BBF Services found with ES_Legacy_ID__c - nothing to true-up")
else:
    # Build mapping: ES Order ID -> BBF Service ID
    es_to_bbf_map = {svc["ES_Legacy_ID__c"]: svc["Id"] for svc in bbf_services}

    # Step 2: Query ES Orders that don't have BBF_New_Id__c yet
    print("\n📌 Step 2: Query ES Orders missing BBF_New_Id__c...")

    # Build ID list for query (in batches if large)
    es_ids = list(es_to_bbf_map.keys())

    # Query in chunks to avoid SOQL length limits
    chunk_size = 200
    es_orders_to_update = []

    for i in range(0, len(es_ids), chunk_size):
        chunk = es_ids[i : i + chunk_size]
        id_str = "','".join(chunk)

        es_query = f"""SELECT Id, BBF_New_Id__c 
        FROM Order 
        WHERE Id IN ('{id_str}') 
        AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')"""

        result = es_sf.query_all(es_query)
        es_orders_to_update.extend(result["records"])

    print(f"   Found {len(es_orders_to_update)} ES Orders needing BBF_New_Id__c update")

    if len(es_orders_to_update) == 0:
        print("\n✅ All ES Orders already have BBF_New_Id__c - no true-up needed")
    else:
        # Step 3: Update ES Orders in batches
        print(f"\n📌 Step 3: Updating ES Orders in batches of {TRUEUP_BATCH_SIZE}...")

        trueup_success = 0
        trueup_failed = 0
        total = len(es_orders_to_update)
        num_batches = (total + TRUEUP_BATCH_SIZE - 1) // TRUEUP_BATCH_SIZE

        for batch_num in range(num_batches):
            start_idx = batch_num * TRUEUP_BATCH_SIZE
            end_idx = min(start_idx + TRUEUP_BATCH_SIZE, total)
            batch = es_orders_to_update[start_idx:end_idx]

            print(
                f"   Batch {batch_num + 1}/{num_batches}: records {start_idx + 1}-{end_idx}...",
                end=" ",
            )

            batch_success = 0
            for es_order in batch:
                es_id = es_order["Id"]
                bbf_id = es_to_bbf_map.get(es_id)

                if bbf_id:
                    try:
                        es_sf.Order.update(es_id, {"BBF_New_Id__c": bbf_id})
                        trueup_success += 1
                        batch_success += 1
                    except Exception as e:
                        trueup_failed += 1
                        print(f"\n      ❌ {es_id}: {e}")

            print(f"✅ {batch_success}/{len(batch)} succeeded")

            # Delay between batches
            if batch_num < num_batches - 1:
                time.sleep(TRUEUP_DELAY_SECONDS)

        print()
        print(f"✅ True-up complete: {trueup_success} ES Orders updated")
        if trueup_failed > 0:
            print(f"❌ Failed to update: {trueup_failed} ES Orders")


TRUE-UP: SYNC ES BBF_New_Id__c FROM BBF ES_Legacy_ID__c

📌 Step 1: Query BBF Services with ES_Legacy_ID__c...
   Found 8645 BBF Services with ES_Legacy_ID__c

📌 Step 2: Query ES Orders missing BBF_New_Id__c...
   Found 4636 ES Orders needing BBF_New_Id__c update

📌 Step 3: Updating ES Orders in batches of 200...
   Batch 1/24: records 1-200... ✅ 200/200 succeeded
   Batch 2/24: records 201-400... ✅ 200/200 succeeded
   Batch 3/24: records 401-600... ✅ 200/200 succeeded
   Batch 4/24: records 601-800... ✅ 200/200 succeeded
   Batch 5/24: records 801-1000... ✅ 200/200 succeeded
   Batch 6/24: records 1001-1200... ✅ 200/200 succeeded
   Batch 7/24: records 1201-1400... ✅ 200/200 succeeded
   Batch 8/24: records 1401-1600... ✅ 200/200 succeeded
   Batch 9/24: records 1601-1800... ✅ 200/200 succeeded
   Batch 10/24: records 1801-2000... ✅ 200/200 succeeded
   Batch 11/24: records 2001-2200... ✅ 200/200 succeeded
   Batch 12/24: records 2201-2400... 
      ❌ 801Rn00000Y7SLmIAN: Malformed re

In [ ]:
# === UPDATE SERVICES WITH ACCOUNT FROM BAN ===
# After insert, populate Account__c on Services using BAN's Account relationship

print("\n" + "=" * 80)
print("UPDATING SERVICES WITH ACCOUNT FROM BAN")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No Services to update with Account")
    account_update_results = []
else:
    # Query the newly inserted Services with their BAN's Account
    print("Querying migrated Services with BAN Account relationship...")

    query = """SELECT Id, Billing_Account_Number__c, Billing_Account_Number__r.Account__c, Account__c
    FROM Service__c
    WHERE ES_Legacy_ID__c != null AND Account__c = null"""

    result = bbf_sf.query_all(query)
    services_to_update = result["records"]

    print(f"Found {len(services_to_update)} Services needing Account update")

    # Build update records - set Account__c from BAN relationship
    account_updates = []
    for svc in services_to_update:
        ban_account = None
        if svc.get("Billing_Account_Number__r"):
            ban_account = svc["Billing_Account_Number__r"].get("Account__c")

        if ban_account:
            account_updates.append({"Id": svc["Id"], "Account__c": ban_account})

    if len(account_updates) == 0:
        print("⚠️  No Services have BAN with Account relationship")
        account_update_results = []
    else:
        print(f"Updating {len(account_updates)} Services with Account__c...")

        try:
            account_update_results = bbf_sf.bulk.Service__c.update(account_updates)

            success_count = sum(1 for r in account_update_results if r["success"])
            error_count = sum(1 for r in account_update_results if not r["success"])

            print(f"\n✅ Successfully updated: {success_count} Services with Account")
            print(f"❌ Failed to update: {error_count} Services")

            if error_count > 0:
                print("\nFirst 5 update failures:")
                fail_count = 0
                for i, r in enumerate(account_update_results):
                    if not r["success"] and fail_count < 5:
                        print(f"  - {account_updates[i]['Id']}: {r['errors']}")
                        fail_count += 1

        except Exception as e:
            print(f"❌ Error during Account update: {e}")
            account_update_results = []

In [13]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

status_colors = {
    "Success": "C6EFCE",
    "Failed": "FFC7CE",
    "Skipped": "FFEB9C",
}

# --- SHEET 1: Migration Results ---
headers1 = ["ES Order ID", "BBF Service ID", "BBF BAN ID", "Status", "Error"]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "BBF_BAN_ID": item["bbf_ban_id"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "BBF_BAN_ID": item["bbf_ban_id"],
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )
for item in skipped_no_bbf_ban:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "BBF_BAN_ID": "",
            "Status": "Skipped",
            "Error": "No BBF BAN ID found",
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append([r["ES_ID"], r["BBF_ID"], r["BBF_BAN_ID"], r["Status"], r["Error"]])
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)
ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Service Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A6"].font = Font(bold=True)
ws2["B6"].font = Font(bold=True)
ws2.append(["Total Services Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Skipped (No BBF BAN)", len(skipped_no_bbf_ban)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Order ID", "BBF Service ID", "BBF BAN ID"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["bbf_ban_id"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)
ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Order ID", "BBF BAN ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append([item["es_id"], item["bbf_ban_id"], str(item["errors"])])

for item in skipped_no_bbf_ban:
    ws4.append([item["es_id"], "", "Skipped: No BBF BAN ID found"])

for col in ws4.columns:
    col_cells = list(col)
    max_length = max(len(str(cell.value)) for cell in col_cells) if col_cells else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)
ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} Services)")
print(f"   📈 Sheet 2: Summary")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} mappings)")
print(
    f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts) + len(skipped_no_bbf_ban)} failures)"
)


CREATING EXCEL OUTPUT

✅ Excel output saved to: es_bbf_service_migration_20260123_120308.xlsx
   📊 Sheet 1: Migration Results (8628 Services)
   📈 Sheet 2: Summary
   🔗 Sheet 3: ID Mapping (8628 mappings)
   ⚠️  Sheet 4: Failed Inserts (0 failures)


In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Order queried: {len(es_orders)}")
print(f"BBF Service__c inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_orders)*100:.1f}%"
    if len(es_orders) > 0
    else "N/A - No Services processed"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} Services.")
    print("   To migrate ALL Services, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Service migration finished.")
    print("   Next: Migrate Service_Charge__c (OrderItem → Service_Charge__c)")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} Services failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")

if len(skipped_no_bbf_ban) > 0:
    print(
        f"\n⚠️  {len(skipped_no_bbf_ban)} Services skipped (no BBF BAN - MASTER-DETAIL REQUIRED)"
    )
    print("   These Orders must have their parent BAN migrated first")

---
## Next Steps: Service_Charge__c Migration

After Service migration is complete, use the **ID Mapping sheet** from this Excel output to migrate Service_Charge__c:

### Prerequisites for Service_Charge__c Migration
| Prerequisite | Status | Notes |
|--------------|--------|-------|
| Account | ✅ Complete | Account migration done |
| BAN__c | ✅ Complete | BAN migration done |
| Contact | ✅ Complete | Contact migration done |
| Location__c | ✅ Complete | Location migration done |
| Service__c | 🔄 This notebook | Use ID Mapping sheet |

### ID Mapping Files Needed for Service_Charge__c
- `es_bbf_service_migration_*.xlsx` → Service ID mapping (this file)

### ES OrderItem → BBF Service_Charge__c
- Master-Detail: Service_Charge__c.Service__c → Service__c.Id
- Query: OrderItem WHERE Order.BBF_New_Id__c != null
- Map: OrderItem.OrderId → ES Order.BBF_New_Id__c (BBF Service ID)

## Field Mapping Reference

### ES Order → BBF Service__c Field Mapping (Day 1 Minimum)

| ES Field | BBF Field | Notes |
|----------|-----------|-------|
| Id | ES_Legacy_ID__c | Tracking |
| Service_ID__c / Name / OrderNumber | Name | Service identifier |
| Billing_Invoice__r.BBF_New_Id__c | Billing_Account_Number__c | MASTER-DETAIL (REQUIRED) |
| Account.BBF_New_Id__c | Account__c | Lookup to Account |
| Address_A__r.BBF_New_Id__c | A_Location__c | Lookup to Location |
| Address_Z__r.BBF_New_Id__c | Z_Location__c | Lookup to Location (optional) |
| (default False) | TSP__c | Required boolean |
| (default 'EVS') | busUnit__c | Required picklist |
| Status | Service_Status__c | Picklist mapping |
| Description | Service_Description__c | Text |
| Circuit_ID__c | Circuit_ID__c | Direct map |
| Billing_Start_Date__c | Billing_Start_Date__c | Date |
| MRC__c | MRC__c | Currency |
| NRC__c | NRC__c | Currency |
| Bandwidth_mbps__c | Bandwidth__c | Number |
| Term_Months__c | Contract_Term_Months__c | Number |

## Cleanup Apex (if needed)

### Delete Migrated Services from BBF
```apex
List<Service__c> services = [SELECT Id, Name FROM Service__c WHERE ES_Legacy_ID__c != null];
System.debug('Found ' + services.size() + ' migrated Services');
delete services;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<Order> orders = [SELECT Id, BBF_New_Id__c 
                      FROM Order 
                      WHERE BBF_New_Id__c != NULL];
System.debug('Found ' + orders.size() + ' records to reset');
for (Order ord : orders) {
    ord.BBF_New_Id__c = NULL;
}
update orders;
```